# 01 - BigQuery - Джерело даних для таблиці
Використовуйте BigQuery для завантаження та підготовки даних для машинного навчання:

**Передумови:**
- [00 - Налаштування середовища](00%20-%20Environment%20Setup.ipynb)

**Ресурси:**
- [Клієнт Python для Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
- [Завантажити дані BigQuery до Pandas](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)
- [Блокноти з шаблонами BigQuery](https://github.com/GoogleCloudPlatform/bigquery-notebooks/tree/main/notebooks/official/template_notebooks)
<!-- - Використання BigQuery з Python, блокноти у цьому репозиторії:
    - [Tips/BigQuery - Python Client](../Tips/BigQuery%20-%20Python%20Client.ipynb)
    - [03 - BigQuery ML (BQML)/Вступ до BigQuery ML (BQML)](../03%20-%20BigQuery%20ML%20(BQML)/Introduction%20to%20BigQuery%20ML%20(BQML).ipynb)
    - [Прикладне прогнозування/1 - Огляд та підготовка даних для прогнозування часових рядів BigQuery](../Applied%20Forecasting/1%20-%20BigQuery%20Time%20Series%20Forecasting%20Data%20Review%20and%20Preparation.ipynb) -->

---
## Вхідні дані

**Огляд**

Цей блокнот імпортує дані для поточного проекту в Google BigQuery. Всі інші блокноти використовують BigQuery як джерело даних.

Цей блокнот, `01 - BigQuery - Table Data Source`, розпочинає життєвий цикл машинного навчання, імпортуючи вихідні дані та готуючи їх до машинного навчання.

Подальші процеси використовують табличні дані, щоб вирішувати задачі навчання з вчителем: передбачити цільову змінну, використовуючи стовпчики ознак. Задачу яку будемо вирішувати - класифікація.

**Дані

Вихідні дані спочатку експортуються до хмарного сховища Google у форматі CSV. Вихідною таблицею BigQuery є `bigquery-public-data.ml_datasets.ulb_fraud_detection`.  Це таблиця транзакцій за кредитними картками, які класифікуються як шахрайські, `Class = 1`, або звичайні `Class = 0`.    
- Дані можна додатково дослідити за цим [Kaggle-покликанням](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).
- Більше про публічні набори даних BigQuery [тут](https://cloud.google.com/bigquery/public-data)

**Опис даних**

Це таблиця з 284 807 транзакцій за кредитними картками, класифікованих як фрод або нормальні в колонці `Class`.  З метою захисту конфіденційності вихідні ознаки були перетворені за допомогою [аналізу головних компонент (PCA)] (https://en.wikipedia.org/wiki/Principal_component_analysis) у 28 ознак з іменами `V1, V2, ... V28` (з плаваючою комою).  Дві описові ознаки подано без перетворення за допомогою PCA:
- `Time` (int) - секунди, що пройшли між транзакцією та найпершою транзакцією в таблиці
- `Amount` (float) - сума транзакції

>**Коротка довідка про PCA**<p>PCA - це метод навчання без вчителя: у ньому немає цільової змінної.  PCA зазвичай використовується як метод зменшення змінних/ознак.  Якщо у вас є 100 ознак, ви можете звести їх до числа p (скажімо, 10) прогнозованих ознак.  Вибір цього числа є балансом між тим, наскільки добре воно може пояснити дисперсію повного простору ознак і зменшенням кількості ознак.  Кожна спроектована ознака є ортогональною до іншої ознаки, що означає відсутність кореляції між цими новими спроектованими ознаками.</p>

**Підготовка даних**

Цей блокнот додає два стовпці до даних і зберігає їх у новій таблиці з суфіксом `_prepped`.  
- `transaction_id` (рядок) унікальний ідентифікатор для рядка/транзакції
- `splits` (рядок) розділяє транзакції на набори для `TRAIN` (80%), `VALIDATE` (10%) і `TEST` (10%)

---
## Налаштування

вхідні дані:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
REGION = 'us-central1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

пакети:

In [ ]:
from google.cloud import bigquery
from google.cloud import storage

клієнти:

In [ ]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

параметри:

In [ ]:
BUCKET = PROJECT_ID

---
## Збережіть вихідні дані в GCS Storage Bucket
Перевірте, чи існує таблиця, і створіть її, якщо не має:
- експорт з таблиці bigquery в GCS bucket у форматі CSV
    - посилання на таблицю міститься у змінній `BQ_SOURCE` у верхній частині цього блокнота
- [Експорт табличних даних](https://cloud.google.com/bigquery/docs/exporting-data#python)
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

In [ ]:
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

In [ ]:
bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'Файл вже створено за адресою: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Експортуємо...')
    extract.result()
    print(f'Дані експортовані за адресою: gs://{bucketDef.name}/{file}')

перелічити файли у бакеті:

In [ ]:
list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}'))

In [ ]:
print(f'Ознайомитися з файлами в консолі можна тут:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')

---
## Створення BigQuery Dataset

Перелічіть набори даних BigQuery в проекті:

In [ ]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

Створіть набір даних, якщо він відсутній:

In [ ]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

Перелічіть набори даних BigQuery в проекті:

In [ ]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

---
## Створення таблиці BigQuery
- імпортуємо дані з бакету
- [Завантаження CSV даних з хмарного сховища](https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv)
- [BigQuery Python Client](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.client.Client#google_cloud_bigquery_client_Client_extract_table)

In [ ]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'Таблиця вже існує: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Створюємо таблицю ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Завершено створення таблиці: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

### Отримання та перегляд вибірки з таблиці:
> **Примітка:** Оператор `LIMIT 5` обмежує кількість рядків, що повертаються BigQuery, до 5, але BigQuery все одно виконує повне сканування таблиці.  Якщо у вас таблиця більша за 1 ГБ і ви хочете обмежити кількість рядків, що скануються для швидкого перегляду, то ефективнішою буде заміна `LIMIT 5` на `TABLESAMPLE SYSTEM (1 PERCENT)`.  Для таблиць розміром менше 1 ГБ програма все одно повертатиме повну таблицю.  Докладніше про [тут](https://cloud.google.com/bigquery/docs/table-sampling)

In [ ]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

### Перевірте цю таблицю в консолі BigQuery Console:
- Натисніть: https://console.cloud.google.com/bigquery
- Переконайтеся, що вибрано правильний проєкт
- У розділі Explore розгорніть цей проект і перегляньте набір даних і таблицю

In [ ]:
print(f"Пряме посилання на цей проект у BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

---
## Перегляд даних у BigQuery
Для перегляду даних можна використовувати додаткові SQL-запити.  У цьому розділі показано переміщення таблиці до фрейму даних Pandas для локального перегляду у Python:

> **Примітка:** <p>Цей запит вибирає лише один стовпець.  Це означає, що BigQuery сканує менше даних, оскільки не обробляє інші стовпці.  </p>

In [ ]:
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [ ]:
df['Class'].value_counts()

In [ ]:
df['Class'].value_counts(normalize=True)

---
## Підготовка даних для аналізу

Створіть заздалегідь підготовлену версію даних з тестовим/тренувальним розбиттям за допомогою SQL DDL:

In [ ]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [ ]:
(job.ended-job.started).total_seconds()

In [ ]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

Перегляньте розподіл між навчальною та тестовою вибірками:

In [ ]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

Отримати підмножину даних до фрейму даних Pandas:

In [ ]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [ ]:
data.head()

---
## Видалити ресурси
див. зошит [_Cleanup](_Cleanup.ipynb)